In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

## Load Data

In [3]:
account = pd.read_csv('data/account.csv', encoding='ISO-8859-1')
concerts_1415=pd.read_csv('data/concerts_2014-15.csv')
concerts=pd.read_csv('data/concerts.csv')
sample_submission=pd.read_csv('data/sample_submission.csv')
subscriptions=pd.read_csv('data/subscriptions.csv')
test=pd.read_csv('data/test.csv')
tickets_all=pd.read_csv('data/tickets_all.csv')
train=pd.read_csv('data/train.csv')
zipcodes=pd.read_csv('data/zipcodes.csv')

#### Merge Subscription and Construct Train Data

In [4]:
# Handle subscription data
def handle_subscription(group):
    # Safely handle NaN values in cities
    shipping_city = group['shipping.city'].fillna("")
    billing_city = group['billing.city'].fillna("")

    # If there are any subscription data
    if group['season'].notna().any():
        return pd.DataFrame({

            # Total season:
            'total_season': [group.shape[0]],

            # Package features
            'full_package': [(group['package'] == 'Full').sum()],
            'quartet_package': [(group['package'].isin(['Quartet', 'Quartet A', 'Quartet B', 'Quartet CC'])).sum()],
            'trio_package': [(group['package'].isin(['Trio', 'Trio A', 'Trio B'])).sum()],
            'cyo_package': [(group['package'] == 'CYO').sum()],
            'full_upgrade_package': [(group['package'] == 'Full upgrade').sum()],

            # Seat features
            'total_seats': [group['no.seats'].sum()],

            # Location features
            'location_num': [group['location'].nunique()],
            'location_near_resident_sum': [(group['location'] == shipping_city).sum() if shipping_city.any() else (group['location'] == billing_city).sum()],
            'location_not_resident_num': [group.shape[0] - (group['location'] == shipping_city).sum() if shipping_city.any() else group.shape[0] - (group['location'] == billing_city).sum()],

            # Section features
            'section_type_numbers': [group['section'].nunique()],
            'premium_orchestra_number': [(group['section'] == 'Premium Orchestra').sum()],
            'orchestra_number': [(group['section'].isin(['Orchestra', 'Orchestra Front', 'Orchestra Rear'])).sum()],
            'balcony_number': [(group['section'].isin(['Balcony Front', 'Balcony Rear', 'Balcony', 'Santa Rosa'])).sum()],
            'dress_circle_number': [(group['section'] == 'Dress Circle').sum()],
            'Gallery_number': [(group['section'] == 'Gallery').sum()],
            'Box_number': [(group['section'].isin(['Box', 'Box House Left', 'Box House Right'])).sum()],
            'floor_number': [(group['section'] == 'Floor').sum()],

            # Price features (some seasons do not have price level data using another feature to present it and calculate mean without them)
            'mean_price_level': [-1 if (~group['price.level'].isna()).sum()==0 else group[~group['price.level'].isna()]['price.level'].mean()],

            'non_price_level_subscription': [group['season'].isin(['2002-2003','2003-2004','2004-2005']).sum()],

            # Subscription tier features
            'mean_subscription_tier': [group['subscription_tier'].mean()],
            'multiple_subs_number': [(group['multiple.subs'] == 'yes').sum()],

            # other features
            'account.id': [group['account.id'].iloc[0]],
            'label': [group['label'].iloc[0]] if 'label' in group.columns else [-1],
            'shipping.zip.code': [group['shipping.zip.code'].iloc[0]],
            'billing.zip.code': [group['billing.zip.code'].iloc[0]],
            'shipping_city': [shipping_city.iloc[0]],
            'billing_city': [billing_city.iloc[0]],
            'relationship': [group['relationship'].iloc[0]],
            'amount.donated.2013': [group['amount.donated.2013'].iloc[0]],
            'amount.donated.lifetime': [group['amount.donated.lifetime'].iloc[0]],
            'no.donations.lifetime': [group['no.donations.lifetime'].iloc[0]],
            'first.donated': [group['first.donated'].iloc[0]],
        })
    
    # no subscription data, just fill with -1
    else:
        return pd.DataFrame({
            # Total season:
            'total_season': [-1],

            # Package features
            'full_package': [-1],
            'quartet_package': [-1],
            'trio_package': [-1],
            'cyo_package': [-1],
            'full_upgrade_package': [-1],

            # Seat features
            'total_seats': [-1],

            # Location features
            'location_num': [-1],
            'location_near_resident_sum': [-1],
            'location_not_resident_num': [-1],

            # Section features
            'section_type_numbers': [-1],
            'premium_orchestra_number': [-1],
            'orchestra_number': [-1],
            'balcony_number': [-1],
            'dress_circle_number': [-1],
            'Gallery_number': [-1],
            'Box_number': [-1],
            'floor_number': [-1],

            # Price features
            'mean_price_level': [-1],

            'non_price_level_subscription': [-1],

            # Subscription tier features
            'mean_subscription_tier': [-1],
            'multiple_subs_number': [-1],

            # other features
            'account.id': [group['account.id'].iloc[0]],
            
            'label': [group['label'].iloc[0]] if 'label' in group.columns else [-1],
            
            'shipping.zip.code': [group['shipping.zip.code'].iloc[0]],
            'billing.zip.code': [group['billing.zip.code'].iloc[0]],
            'shipping_city': [shipping_city.iloc[0]],
            'billing_city': [billing_city.iloc[0]],
            'relationship': [group['relationship'].iloc[0]],
            'amount.donated.2013': [group['amount.donated.2013'].iloc[0]],
            'amount.donated.lifetime': [group['amount.donated.lifetime'].iloc[0]],
            'no.donations.lifetime': [group['no.donations.lifetime'].iloc[0]],
            'first.donated': [group['first.donated'].iloc[0]],
        })


In [5]:
# Merge train with account
train_merged = pd.merge(train, account, on='account.id', how='left')

# Merge train with subscriptions
train_merged = pd.merge(train_merged, subscriptions, on='account.id', how='left')

# Apply the function using groupby
train_merged = train_merged.groupby('account.id',group_keys=False).apply(handle_subscription)

train_merged=train_merged.reset_index(drop=True)


C:\Users\24549\AppData\Local\Temp\ipykernel_27100\569496751.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_merged = train_merged.groupby('account.id',group_keys=False).apply(handle_subscription)


#### Merge Tickets 

In [36]:
tickets_all['multiple.tickets'].isna().sum()

0

In [37]:
other_columns = train_merged.columns

def handle_tickets(group):

    if group['season'].notna().any():
        return pd.DataFrame({

            # tickets sum
            'sum_tickets': [group['no.seats'].sum()],

            # price features
            'average_price_level': [group['price.level'].mean()],
            
            # seats number features

            'total_seats': [group['no.seats'].sum()],

            # localtion features
            'location_num': [group['location'].nunique()],
            'location_near_resident_sum': [(group['location'] == group['shipping.city']).sum() if group['shipping.city'].notna().any() else (group['location'] == group['billing.city']).sum()],
            'location_not_resident_num': [group.shape[0] - (group['location'] == group['shipping.city']).sum() if group['shipping.city'].notna().any() else group.shape[0] - (group['location'] == group['billing.city']).sum()],
            
            # set features

            'set sum': [group['set'].sum()],

            # multiple_tickets features
            'multiple_tickets_num': [(group['multiple.tickets'] == 'yes').sum()],
            
            # All other features in other_columns



            
            


        })
    
    else:
        return pd.DataFrame({
            # tickets sum
            'sum_tickets': -1,

            # price features
            'average_price_level': -1,
            
            # seats number features

            'total_seats': -1,

            # localtion features
            'location_num': -1,
            'location_near_resident_sum': -1,
            'location_not_resident_num': -1,
            
            # set features

            'set sum': -1,

            # multiple_tickets features
            'multiple_tickets_num': -1,


        })

In [6]:
# to calculate the mean for the set column, first fillna with the mean of the value
train_merged['set']=train_merged.groupby('account.id')['set'].transform(lambda x: x.fillna(x.mean()))

# merge train with tickets all

train_merged = pd.merge(train_merged,tickets_all,on='account.id',how='left')

train_merged = train_merged.groupnby('account.id',group_keys=False).apply(handle_tickets)

train_merged=train_merged.reset_index(drop=True)

,total_season,full_package,quartet_package,trio_package,cyo_package,full_upgrade_package,total_seats,location_num,location_near_resident_sum,location_not_resident_num,...,amount.donated.lifetime,no.donations.lifetime,first.donated,price.level,no.seats,marketing.source,season,location,set,multiple.tickets
0,-1,-1,-1,-1,-1,-1,-1.0,-1,-1,-1,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,11,0,0,0,0,20.0,1,11,0,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21,21,0,0,0,0,39.0,1,0,21,...,11913.0,20,1991-07-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,10,3,0,0,0,25.0,2,0,13,...,2570.0,15,1997-04-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,16,0,0,3,0,35.0,1,0,19,...,5929.0,26,1989-07-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7181,-1,-1,-1,-1,-1,-1,-1.0,-1,-1,-1,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7182,-1,-1,-1,-1,-1,-1,-1.0,-1,-1,-1,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7183,-1,-1,-1,-1,-1,-1,-1.0,-1,-1,-1,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7184,-1,-1,-1,-1,-1,-1,-1.0,-1,-1,-1,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Missing Values,Duplicates and Outliers



##### Description

In [7]:
train_merged[train_merged['mean_price_level'].isna()]['account.id']

Series([], Name: account.id, dtype: object)

In [8]:
train_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7186 entries, 0 to 7185
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   total_season                  7186 non-null   int64  
 1   full_package                  7186 non-null   int64  
 2   quartet_package               7186 non-null   int64  
 3   trio_package                  7186 non-null   int64  
 4   cyo_package                   7186 non-null   int64  
 5   full_upgrade_package          7186 non-null   int64  
 6   total_seats                   7186 non-null   float64
 7   location_num                  7186 non-null   int64  
 8   location_near_resident_sum    7186 non-null   int64  
 9   location_not_resident_num     7186 non-null   int64  
 10  section_type_numbers          7186 non-null   int64  
 11  premium_orchestra_number      7186 non-null   int64  
 12  orchestra_number              7186 non-null   int64  
 13  bal

##### Duplicates

In [9]:
# Delete Duplicate Rows
train_merged=train_merged.drop_duplicates()

# Delete shipping_city and billing_city because shipping_city is the same as shipping zip code and billing_city is the same as billing zip code

train_merged=train_merged.drop(['shipping_city','billing_city'],axis=1)

##### Missing Values


In [10]:
train_merged.isnull().sum()

total_season                       0
full_package                       0
quartet_package                    0
trio_package                       0
cyo_package                        0
full_upgrade_package               0
total_seats                        0
location_num                       0
location_near_resident_sum         0
location_not_resident_num          0
section_type_numbers               0
premium_orchestra_number           0
orchestra_number                   0
balcony_number                     0
dress_circle_number                0
Gallery_number                     0
Box_number                         0
floor_number                       0
mean_price_level                   0
non_price_level_subscription       0
mean_subscription_tier             0
multiple_subs_number               0
account.id                         0
label                              0
shipping.zip.code               7088
billing.zip.code                 980
relationship                    6961
a

In [11]:
# Fill NA with empty string
train_merged['shipping.zip.code']=train_merged['shipping.zip.code'].fillna("")
train_merged['billing.zip.code']=train_merged['billing.zip.code'].fillna("")
train_merged['relationship']=train_merged['relationship'].fillna("")
train_merged['first.donated']=train_merged.apply(lambda x: '1800-01-01 00:00:00' if x['no.donations.lifetime']==0 else (0 if pd.isna(x['first.donated']) else x['first.donated']),axis=1)

train_merged.head(5)

,total_season,full_package,quartet_package,trio_package,cyo_package,full_upgrade_package,total_seats,location_num,location_near_resident_sum,location_not_resident_num,...,amount.donated.lifetime,no.donations.lifetime,first.donated,price.level,no.seats,marketing.source,season,location,set,multiple.tickets
0,-1,-1,-1,-1,-1,-1,-1.0,-1,-1,-1,...,0.0,0,1800-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,11,0,0,0,0,20.0,1,11,0,...,0.0,0,1800-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21,21,0,0,0,0,39.0,1,0,21,...,11913.0,20,1991-07-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,10,3,0,0,0,25.0,2,0,13,...,2570.0,15,1997-04-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,16,0,0,3,0,35.0,1,0,19,...,5929.0,26,1989-07-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Turn first donated to timestamp data

In [12]:
train_merged['first.donated'] = pd.to_datetime(train_merged['first.donated'])

train_merged.head(5)

,total_season,full_package,quartet_package,trio_package,cyo_package,full_upgrade_package,total_seats,location_num,location_near_resident_sum,location_not_resident_num,...,amount.donated.lifetime,no.donations.lifetime,first.donated,price.level,no.seats,marketing.source,season,location,set,multiple.tickets
0,-1,-1,-1,-1,-1,-1,-1.0,-1,-1,-1,...,0.0,0,1800-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,11,0,0,0,0,20.0,1,11,0,...,0.0,0,1800-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21,21,0,0,0,0,39.0,1,0,21,...,11913.0,20,1991-07-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,10,3,0,0,0,25.0,2,0,13,...,2570.0,15,1997-04-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,16,0,0,3,0,35.0,1,0,19,...,5929.0,26,1989-07-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# check missing values
train_merged.isnull().sum()

total_season                       0
full_package                       0
quartet_package                    0
trio_package                       0
cyo_package                        0
full_upgrade_package               0
total_seats                        0
location_num                       0
location_near_resident_sum         0
location_not_resident_num          0
section_type_numbers               0
premium_orchestra_number           0
orchestra_number                   0
balcony_number                     0
dress_circle_number                0
Gallery_number                     0
Box_number                         0
floor_number                       0
mean_price_level                   0
non_price_level_subscription       0
mean_subscription_tier             0
multiple_subs_number               0
account.id                         0
label                              0
shipping.zip.code                  0
billing.zip.code                   0
relationship                       0
a

#### Feature Encoding

In [14]:
# ## WOE Encoding

import category_encoders as ce

# Use WoE to encode shipping zip code and billing zip code

woe = ce.WOEEncoder([ 'shipping.zip.code', 'billing.zip.code','relationship'])

train_merged['shipping.zip.code'] = train_merged['shipping.zip.code'].astype(str)
train_merged['billing.zip.code'] = train_merged['billing.zip.code'].astype(str)

woe.fit(train_merged[['shipping.zip.code', 'billing.zip.code','relationship']], train_merged['label'])

train_merged[['shipping.zip.code', 'billing.zip.code','relationship']] = woe.transform(train_merged[['shipping.zip.code', 'billing.zip.code','relationship']])

#### Construct More Features

In [15]:
# Calculate the number of days since the first donation

now = pd.to_datetime('2014-09-01')

train_merged['days_since_first_donation'] = (now - train_merged['first.donated']).dt.days

# drop first donated column

train_merged=train_merged.drop(['first.donated'],axis=1)

train_merged.head(5)

,total_season,full_package,quartet_package,trio_package,cyo_package,full_upgrade_package,total_seats,location_num,location_near_resident_sum,location_not_resident_num,...,amount.donated.lifetime,no.donations.lifetime,price.level,no.seats,marketing.source,season,location,set,multiple.tickets,days_since_first_donation
0,-1,-1,-1,-1,-1,-1,-1.0,-1,-1,-1,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78405
1,11,11,0,0,0,0,20.0,1,11,0,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78405
2,21,21,0,0,0,0,39.0,1,0,21,...,11913.0,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8460
3,13,10,3,0,0,0,25.0,2,0,13,...,2570.0,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6346
4,19,16,0,0,3,0,35.0,1,0,19,...,5929.0,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9190


#### TestSet

In [16]:
# change the name

test['account.id']=test['ID']
test.drop(['ID'],axis=1,inplace=True)

In [17]:
# Merge test with account
test_merged = pd.merge(test, account, on='account.id', how='left')

# Merge test with subscriptions
test_merged = pd.merge(test_merged, subscriptions, on='account.id', how='left')

# Apply the function using groupby
test_merged = test_merged.groupby('account.id',group_keys=False).apply(handle_subscription)

test_merged=test_merged.reset_index(drop=True)

test_merged.drop(['label'],axis=1,inplace=True)

C:\Users\24549\AppData\Local\Temp\ipykernel_27100\1953010395.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_merged = test_merged.groupby('account.id',group_keys=False).apply(handle_subscription)


In [18]:
# Delete Duplicate Rows
test_merged=test_merged.drop_duplicates()

# Delete shipping_city and billing_city because shipping_city is the same as shipping zip code and billing_city is the same as billing zip code

test_merged=test_merged.drop(['shipping_city','billing_city'],axis=1)


In [19]:
# Fill NA with empty string
test_merged['shipping.zip.code']=test_merged['shipping.zip.code'].fillna("")
test_merged['billing.zip.code']=test_merged['billing.zip.code'].fillna("")
test_merged['relationship']=test_merged['relationship'].fillna("")
test_merged['first.donated']=test_merged.apply(lambda x: '1800-01-01 00:00:00' if x['no.donations.lifetime']==0 else (0 if pd.isna(x['first.donated']) else x['first.donated']),axis=1)

# change to the type of datetime
test_merged['first.donated'] = pd.to_datetime(test_merged['first.donated'])


In [20]:
# encode

test_merged[['shipping.zip.code', 'billing.zip.code','relationship']] = woe.transform(test_merged[['shipping.zip.code', 'billing.zip.code','relationship']])

In [21]:
# Calculate the number of days since the first donation

now = pd.to_datetime('2014-09-01')

test_merged['days_since_first_donation'] = (now - test_merged['first.donated']).dt.days

# drop first donated column

test_merged=test_merged.drop(['first.donated'],axis=1)

test_merged.head(5)



,total_season,full_package,quartet_package,trio_package,cyo_package,full_upgrade_package,total_seats,location_num,location_near_resident_sum,location_not_resident_num,...,mean_subscription_tier,multiple_subs_number,account.id,shipping.zip.code,billing.zip.code,relationship,amount.donated.2013,amount.donated.lifetime,no.donations.lifetime,days_since_first_donation
0,19,19,0,0,0,0,35.0,1,0,19,...,1.842105,0,001i000000LhyPF,-0.010918,0.855079,0.031042,1500.0,28435.00,28,10286
1,16,15,0,1,0,0,30.0,1,0,16,...,1.812500,0,001i000000LhyPG,-0.010918,-0.275282,0.031042,300.0,4969.00,16,6356
2,16,16,0,0,0,0,29.0,1,0,16,...,1.812500,0,001i000000LhyPP,-0.010918,1.357873,0.031042,0.0,4047.96,21,5752
3,7,0,3,4,0,0,14.0,1,0,7,...,1.000000,0,001i000000LhyPb,-0.010918,1.056946,0.031042,0.0,5600.00,8,2668
4,21,17,4,0,0,0,39.0,2,0,21,...,1.714286,0,001i000000LhyPg,-0.010918,0.735625,0.031042,150.0,3678.00,35,9555


## Train and Test Model

#### Data Processing

In [22]:
test_merged['account.id'].head()

0    001i000000LhyPF
1    001i000000LhyPG
2    001i000000LhyPP
3    001i000000LhyPb
4    001i000000LhyPg
Name: account.id, dtype: object

In [23]:
# Split the data into X and y

X_train=train_merged.drop(['label','account.id'],axis=1)
y_train=train_merged['label']

X_test=test_merged.drop(['account.id'],axis=1)

In [24]:
# Scale data

# Scale train data
scaler = StandardScaler()

numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

categorical_cols = X_train.select_dtypes(include=['object']).columns

train_scaled=X_train.copy()

scaler.fit(train_scaled[numerical_cols])

train_scaled[numerical_cols] = scaler.transform(train_scaled[numerical_cols])

# Scale test data

test_scaled=X_test.copy()

test_scaled[numerical_cols] = scaler.transform(test_scaled[numerical_cols])


KeyError: "['no.seats', 'set'] not in index"

#### Model Training And Prediction

In [ ]:
# import modules

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor as cat

from sklearn.metrics import roc_auc_score

In [ ]:
# Kfold cross validation

splits = 5

kf = KFold(n_splits=splits, shuffle=True, random_state=42)

cat_features = []

def cv_model(clf, x_train, y_train, x_test, clf_name , kf):
    
    cv_scores = []

    test_all = []

    for i, (train_index, valid_index) in enumerate(kf.split(x_train, y_train)):

        print('************************************ {} ************************************'.format(str(i+1)))

        trn_x, trn_y, val_x, val_y = x_train.iloc[train_index], y_train[train_index], x_train.iloc[valid_index], y_train[valid_index]

        # LightGBM
        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            # training parameters
            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 4,
                'num_leaves': 2 ** 4,
                'lambda_l2': 10,
                'feature_fraction': 0.7,
                'bagging_fraction': 0.7,
                'bagging_freq': 10,
                'learning_rate': 0.15,
                'seed': 2022,
                'n_jobs':-1,
                'verbose':-1
            }
            # model training
            model = clf.train(params, train_matrix, 30000, valid_sets=[train_matrix, valid_matrix], 
                              categorical_feature=[])
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(x_test, num_iteration=model.best_iteration)
        
        # XGBoost
        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            test_matrix = clf.DMatrix(x_test)
            
            # training parameters
            params = {'booster': 'gbtree',
                      'objective': 'binary:logistic',
                      'eval_metric': 'auc',
                      'gamma': 1,
                      'min_child_weight': 1.5,
                      'max_depth': 7,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'eta': 0.125,
                      'tree_method': 'exact',
                      'seed': 2020,
                      'nthread': 36,
                      "silent": True,
                      }
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]

            # model training
            model = clf.train(params, train_matrix, num_boost_round=50000, evals=watchlist,verbose_eval=True)
            val_pred  = model.predict(valid_matrix)
            test_pred = model.predict(test_matrix)

        # Catboost         
        if clf_name == "cat":
            
            # training parameters
            params = {'learning_rate': 0.134, 
            'depth': 10 ,
            'l2_leaf_reg': 5, 
            'bootstrap_type': 'Bernoulli',
            'od_type': 'Iter', 
            'od_wait': 2000, 
            'random_seed': 164, 
            'allow_writing_files': False
            }

            # model training
            model = clf(iterations=30000, **params)
            model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                      cat_features= cat_features, use_best_model=True, verbose=600)
            
            val_pred  = model.predict(val_x)
            test_pred = model.predict(x_test)
        

        test_all.append(test_pred)

        cv_scores.append(roc_auc_score(val_y, val_pred))
        
        print(cv_scores)

    # output
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))

    # Convert test_all (list of lists) to a NumPy array
    test_all_array = np.array(test_all)

    # Calculate the mean across the lists (axis=0 computes mean element-wise across all lists)
    mean_output = np.mean(test_all_array, axis=0)

    return mean_output
        
def lgb_model(x_train, y_train, x_test):
    lgb_test = cv_model(lgb, x_train, y_train, x_test, "lgb", kf)
    return lgb_test

def xgb_model(x_train, y_train, x_test):
    xgb_test = cv_model(xgb, x_train, y_train, x_test, "xgb", kf)
    return xgb_test

def cat_model(x_train, y_train, x_test):
    cat_test = cv_model(cat, x_train, y_train, x_test, "cat", kf)
    return cat_test

In [ ]:
X_test.head()

,total_season,full_package,quartet_package,trio_package,cyo_package,full_upgrade_package,total_seats,location_num,location_near_resident_sum,location_not_resident_num,...,non_price_level_subscription,mean_subscription_tier,multiple_subs_number,shipping.zip.code,billing.zip.code,relationship,amount.donated.2013,amount.donated.lifetime,no.donations.lifetime,days_since_first_donation
0,19,19,0,0,0,0,35.0,1,0,19,...,3,1.842105,0,-0.013762,0.940593,0.031785,1500.0,28435.00,28,10286
1,16,15,0,1,0,0,30.0,1,0,16,...,2,1.812500,0,-0.013762,-0.192705,0.031785,300.0,4969.00,16,6356
2,16,16,0,0,0,0,29.0,1,0,16,...,3,1.812500,0,-0.013762,1.467478,0.031785,0.0,4047.96,21,5752
3,7,0,3,4,0,0,14.0,1,0,7,...,0,1.000000,0,-0.013762,1.021739,0.031785,0.0,5600.00,8,2668
4,21,17,4,0,0,0,39.0,2,0,21,...,3,1.714286,0,-0.013762,0.610443,0.031785,150.0,3678.00,35,9555


In [ ]:
# lgb_test=lgb_model(X_train, y_train, X_test)

In [ ]:
# xgb_test=xgb_model(X_train, y_train, X_test)

In [ ]:
cat_test=cat_model(X_train, y_train, X_test)

************************************ 1 ************************************
0:	learn: 0.1965540	test: 0.2275599	best: 0.2275599 (0)	total: 154ms	remaining: 1h 16m 53s
600:	learn: 0.0519936	test: 0.1408650	best: 0.1398162 (256)	total: 5.45s	remaining: 4m 26s
1200:	learn: 0.0477067	test: 0.1414469	best: 0.1398162 (256)	total: 10.8s	remaining: 4m 19s
1800:	learn: 0.0469589	test: 0.1417723	best: 0.1398162 (256)	total: 16.2s	remaining: 4m 13s
Stopped by overfitting detector  (2000 iterations wait)

bestTest = 0.1398161729
bestIteration = 256

Shrink model to first 257 iterations.
[0.9671912059843095]
************************************ 2 ************************************
0:	learn: 0.2011978	test: 0.2005215	best: 0.2005215 (0)	total: 5.9ms	remaining: 2m 57s
600:	learn: 0.0561384	test: 0.1367520	best: 0.1342748 (70)	total: 5.28s	remaining: 4m 18s
1200:	learn: 0.0525236	test: 0.1381245	best: 0.1342748 (70)	total: 10.6s	remaining: 4m 15s
1800:	learn: 0.0517360	test: 0.1385321	best: 0.134274

## Output Data to Final Submission

In [ ]:
# merge the results to the test merged

# I choose catboost for the final model

output=pd.DataFrame({'ID':test_merged['account.id'],'Predicted':cat_test})

output.to_csv('submission.csv',index=False)

